In [3]:
import cv2
import numpy as np

src = cv2.imread("coins.jpg")
src = cv2.resize(src, None, fx=0.2, fy=0.2)
cv2.imshow("src", src)

# 影線轉成灰階
src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
# # 二值化處理影像
ret, dst_binary = cv2.threshold(src_gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
# 使用自適應閾值進行二值化
# dst_binary = cv2.adaptiveThreshold(src_gray, 255,
#                                    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
#                                    cv2.THRESH_BINARY_INV, 11, -2)
# 找尋影像內的輪廓
contours, hierarchy = cv2.findContours(dst_binary,
                                       cv2.RETR_EXTERNAL,
                                       cv2.CHAIN_APPROX_SIMPLE)
# 繪製圖形輪廓
dst = cv2.drawContours(src, contours, -1, (0, 255, 0), 5)
# 初始化統計數據
triangle_count = 0                  # 三角形計數器
rectangle_count = 0                 # 矩形計數器
circle_count = 0                    # 圓形計數器

# 輪廓處理
for i, contour in enumerate(contours):
    # 輪廓近似
    epsilon = 0.02 * cv2.arcLength(contour, True)   # 根據輪廓周長調整
    approx = cv2.approxPolyDP(contour, epsilon, True)

    # 輪廓點數量
    num_points = len(approx)
    # 判斷類型
    if num_points == 3:                             # 三角形
        triangle_count += 1
        print(f"輪廓 {i} : 三角形, 輪廓點數量 = {num_points}")
    elif num_points == 4:                           # 矩形
        print(f"輪廓 {i} : 矩形,   輪廓點數量 = {num_points}")
        rectangle_count += 1
    elif num_points >= 8:                           # 圓形(近似為園)
        circle_count += 1
        print(f"輪廓 {i} : 圓形,   輪廓點數量 = {num_points}")
    else:
        print(f"輪廓 {i} : 其他形狀, 輪廓點數量 = {num_points}")

# 統計結果
print(f"三角形的輪廓總數量 : {triangle_count}")
print(f"矩形的輪廓總數量   : {rectangle_count}")
print(f"圓形的輪廓總數量   : {circle_count}")


# # 繪製文字
# font = cv2.FONT_HERSHEY_SIMPLEX
# for i in range(len(contours)):
#     cv2.putText(dst, str(i), contours[i][0][0], font, 1, (0, 0, 255), 2)
# for c in contours:                                   # 繪製中心點迴圈
#     M = cv2.moments(c)                               # 影像矩
#     Cx = int(M["m10"] / M["m00"])                    # 質心 x 座標
#     Cy = int(M["m01"] / M["m00"])                    # 質心 y 座標
#     cv2.circle(dst, (Cx, Cy), 5, (255, 0, 0), -1)    # 繪製中心點
cv2.imshow("result", dst)                            # 顯示結果影像
cv2.waitKey(0)
cv2.destroyAllWindows()

輪廓 0 : 其他形狀, 輪廓點數量 = 2
輪廓 1 : 其他形狀, 輪廓點數量 = 1
輪廓 2 : 其他形狀, 輪廓點數量 = 1
輪廓 3 : 其他形狀, 輪廓點數量 = 1
輪廓 4 : 其他形狀, 輪廓點數量 = 2
輪廓 5 : 其他形狀, 輪廓點數量 = 1
輪廓 6 : 其他形狀, 輪廓點數量 = 1
輪廓 7 : 其他形狀, 輪廓點數量 = 2
輪廓 8 : 其他形狀, 輪廓點數量 = 1
輪廓 9 : 其他形狀, 輪廓點數量 = 2
輪廓 10 : 其他形狀, 輪廓點數量 = 2
輪廓 11 : 其他形狀, 輪廓點數量 = 1
輪廓 12 : 其他形狀, 輪廓點數量 = 1
輪廓 13 : 其他形狀, 輪廓點數量 = 1
輪廓 14 : 其他形狀, 輪廓點數量 = 2
輪廓 15 : 其他形狀, 輪廓點數量 = 1
輪廓 16 : 其他形狀, 輪廓點數量 = 7
輪廓 17 : 其他形狀, 輪廓點數量 = 2
輪廓 18 : 其他形狀, 輪廓點數量 = 1
輪廓 19 : 其他形狀, 輪廓點數量 = 2
輪廓 20 : 其他形狀, 輪廓點數量 = 1
輪廓 21 : 其他形狀, 輪廓點數量 = 1
輪廓 22 : 其他形狀, 輪廓點數量 = 2
輪廓 23 : 矩形,   輪廓點數量 = 4
輪廓 24 : 其他形狀, 輪廓點數量 = 1
輪廓 25 : 其他形狀, 輪廓點數量 = 1
輪廓 26 : 其他形狀, 輪廓點數量 = 2
輪廓 27 : 其他形狀, 輪廓點數量 = 1
輪廓 28 : 其他形狀, 輪廓點數量 = 2
輪廓 29 : 其他形狀, 輪廓點數量 = 2
輪廓 30 : 其他形狀, 輪廓點數量 = 1
輪廓 31 : 其他形狀, 輪廓點數量 = 1
輪廓 32 : 其他形狀, 輪廓點數量 = 1
輪廓 33 : 矩形,   輪廓點數量 = 4
輪廓 34 : 其他形狀, 輪廓點數量 = 1
輪廓 35 : 其他形狀, 輪廓點數量 = 1
輪廓 36 : 其他形狀, 輪廓點數量 = 1
輪廓 37 : 其他形狀, 輪廓點數量 = 2
輪廓 38 : 其他形狀, 輪廓點數量 = 1
輪廓 39 : 其他形狀, 輪廓點數量 = 1
輪廓 40 : 其他形狀, 輪廓點數量 = 1
輪廓 41 : 其他形狀, 輪廓點數量 = 2
輪廓

In [2]:
import cv2
import numpy as np

src = cv2.imread("coins.jpg")
src = cv2.resize(src, None, fx=0.2, fy=0.2)
cv2.imshow("src", src)

# 影線轉成灰階
src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
# 1. 影像預處理：使用 CLAHE 增強對比度
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
clahe_img = clahe.apply(src_gray)
# 使用 Otsu's method 進行二值化處理
ret, dst_binary = cv2.threshold(clahe_img, 127, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
# 自適應閾值處理
# dst_binary = cv2.adaptiveThreshold(src_gray, 255,
#                                    cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
#                                    cv2.THRESH_BINARY_INV, 11, 2)
cv2.imshow('Binary Image', dst_binary)
# 尋找影像內的輪廓
contours, hierarchy = cv2.findContours(dst_binary,
                                       cv2.RETR_LIST,
                                       cv2.CHAIN_APPROX_SIMPLE)


# 創建一個空白圖片，用於繪製橢圓
result_img = np.copy(src)

# 遍歷所有輪廓
for contour in contours:
    # 只要點數足夠，就嘗試擬合
    if len(contour) > 5:
        ellipse = cv2.fitEllipse(contour)
        
        # 計算面積
        (center, axes, orientation) = ellipse
        ellipse_area = np.pi * axes[0] * axes[1]

        # 加上判斷式，確保分母不為零
        if ellipse_area > 0:
            # 只檢查輪廓面積是否大於一個很小的值，例如 1
            if cv2.contourArea(contour) > 140:
                # 繪製橢圓，這樣你可以看到所有被擬合出來的圖形
                cv2.ellipse(result_img, ellipse, (0, 255, 0), 5)

# 顯示結果，觀察所有被擬合出來的「橢圓」
cv2.imshow('All Fitted Ellipses', result_img)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [1]:
import cv2
import numpy as np

def process_coins(image_path):
    # 讀取影像
    src = cv2.imread(image_path)
    if src is None:
        print(f"錯誤：無法讀取影像，請確認檔案路徑: {image_path}")
        return

    # 使用 cv2.resize() 將影像縮小
    # fx 和 fy 分別代表水平和垂直方向的縮放比例
    scale_factor = 1
    scaled_src = cv2.resize(src, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_AREA)

    output = scaled_src.copy()

    # 預處理：灰階和高斯模糊
    gray = cv2.cvtColor(scaled_src, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (9, 9), 2)

    # 由於影像縮小了，圓形半徑的偵測範圍也需要調整
    # 這裡的 minRadius 和 maxRadius 也要根據縮小後的影像來調整
    circles = cv2.HoughCircles(
        blurred,
        cv2.HOUGH_GRADIENT,
        dp=1.2,
        minDist=50,
        param1=50,
        param2=30,
        minRadius=int(20 * scale_factor),  # 調整 minRadius
        maxRadius=int(70 * scale_factor)   # 調整 maxRadius
    )

    total_amount = 0

    if circles is not None:
        circles = np.uint16(np.around(circles))

        # 由於影像縮小了，硬幣的模板半徑也需要按比例調整
        coin_templates = {
            1: (int(30 * scale_factor), int(38 * scale_factor)),  # 1元硬幣的半徑範圍
            5: (int(45 * scale_factor), int(55 * scale_factor)),  # 5元硬幣的半徑範圍
            10: (int(60 * scale_factor), int(70 * scale_factor)) # 10元硬幣的半徑範圍
        }
        recognizer = CoinRecognizer(coin_templates)

        for i in circles[0, :]:
            center_x, center_y, radius = i[0], i[1], i[2]
            value = recognizer.recognize_coin(radius)

            if value > 0:
                total_amount += value
                cv2.circle(output, (center_x, center_y), radius, (0, 255, 0), 2)
                cv2.putText(output, str(value), (center_x - 10, center_y + 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # 在圖片上顯示總金額
    cv2.putText(output, f"Total: {total_amount}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # 顯示結果
    cv2.imshow("Result", output)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


# 你的 CoinRecognizer 類別
class CoinRecognizer:
    def __init__(self, templates):
        self.templates = templates

    def recognize_coin(self, radius):
        for value, (min_r, max_r) in self.templates.items():
            if min_r <= radius <= max_r:
                return value
        return 0

# 調用函式並傳入你的圖片路徑
process_coins("coins3.jpg")

In [1]:
import cv2
import numpy as np

# 建立一個空函式，作為滑動條的回調函式
def nothing(x):
    pass

def process_coins_with_trackbar(image_path):
    # 讀取影像
    src = cv2.imread(image_path)
    if src is None:
        print(f"錯誤：無法讀取影像，請確認檔案路徑: {image_path}")
        return

    # 縮小影像以加快處理速度，並提高圓形檢測準確性
    scale_factor = 0.5  
    scaled_src = cv2.resize(src, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_AREA)

    # 創建一個視窗來顯示調整結果
    cv2.namedWindow('Coin Detection', cv2.WINDOW_AUTOSIZE)
    
    # 創建滑動條，用於調整參數
    cv2.createTrackbar('minRadius', 'Coin Detection', 30, 100, nothing)
    cv2.createTrackbar('maxRadius', 'Coin Detection', 70, 200, nothing)
    cv2.createTrackbar('param1', 'Coin Detection', 50, 200, nothing)
    cv2.createTrackbar('param2', 'Coin Detection', 25, 100, nothing)
    cv2.createTrackbar('minDist', 'Coin Detection', 80, 200, nothing)

    # 預處理：灰階和高斯模糊
    gray = cv2.cvtColor(scaled_src, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (9, 9), 2)
    
    # 將影像轉換為 HSV 色彩空間以利於顏色辨識
    hsv = cv2.cvtColor(scaled_src, cv2.COLOR_BGR2HSV)

    # 啟動迴圈，直到按下 'q' 鍵結束
    while True:
        output = scaled_src.copy()
        
        # 讀取滑動條的值
        min_r = cv2.getTrackbarPos('minRadius', 'Coin Detection')
        max_r = cv2.getTrackbarPos('maxRadius', 'Coin Detection')
        param1 = cv2.getTrackbarPos('param1', 'Coin Detection')
        param2 = cv2.getTrackbarPos('param2', 'Coin Detection')
        minDist = cv2.getTrackbarPos('minDist', 'Coin Detection')
        
        # 進行霍夫圓形變換，使用滑動條的值
        circles = cv2.HoughCircles(
            blurred,
            cv2.HOUGH_GRADIENT,
            dp=1.2,
            minDist=minDist,
            param1=param1,
            param2=param2,
            minRadius=min_r,
            maxRadius=max_r
        )
        
        total_amount = 0

        # 硬幣半徑（像素）模板，請根據你的硬幣實際情況調整
        # 這裡的範圍是基於 scale_factor=0.5 的估計值
        coin_templates = {
            50: (65, 75),
            20: (60, 70), # 20元直徑比50元大一些
            10: (50, 60),
            5: (45, 55),
            1: (40, 50)
        }

        if circles is not None:
            circles = np.uint16(np.around(circles))
            
            for i in circles[0, :]:
                center_x, center_y, radius = i[0], i[1], i[2]
                
                # 獲取硬幣中心的HSV值，用於顏色判斷
                # 確保座標在影像範圍內
                if 0 <= center_y < hsv.shape[0] and 0 <= center_x < hsv.shape[1]:
                    hue_value = hsv[center_y, center_x, 0]
                    value = recognize_coin(radius, hue_value, coin_templates)
                else:
                    value = 0

                if value > 0:
                    total_amount += value
                    cv2.circle(output, (center_x, center_y), radius, (0, 255, 0), 2)
                    cv2.putText(output, str(value), (center_x - 10, center_y + 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
                else:
                    cv2.circle(output, (center_x, center_y), radius, (0, 0, 255), 2)
        
        # 在圖片上顯示總金額
        cv2.putText(output, f"Total: {total_amount}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        
        # 顯示結果
        cv2.imshow('Coin Detection', output)
        
        # 按下 'q' 鍵退出迴圈
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()


def recognize_coin(radius, hue_value, templates):
    # 先判斷是否為金色的50元硬幣
    if 20 <= hue_value <= 40: # 假設黃色色相範圍為20-40
        if templates[50][0] <= radius <= templates[50][1]:
            return 50
            
    # 如果不是50元，再根據半徑範圍判斷其他幣值
    for value, (min_r, max_r) in templates.items():
        if value != 50:
            if min_r <= radius <= max_r:
                return value
                
    return 0


# 調用函式並傳入你的圖片路徑
process_coins_with_trackbar("coins3.jpg")

In [ ]:
import cv2
import numpy as np

# 硬幣幣值與顏色的對應字典 (BGR 格式)
COIN_COLORS = {
    1: (0, 0, 255),    # 1元硬幣 (紅色)
    5: (0, 255, 0),    # 5元硬幣 (綠色)
    10: (255, 0, 0),   # 10元硬幣 (藍色)
    50: (0, 255, 255),  # 50元硬幣 (黃色)
    20: (255, 0, 255),  # 20元硬幣 (紫色)
}

# 幣值與面積範圍的對應字典
# 這些值需要根據你的影像解析度和硬幣實際大小進行校準
# 建議你先執行一次程式，並將輪廓的面積印出，再設定這些值
COIN_AREAS = {
    1: (1500, 2000),    # 1元硬幣面積範圍
    5: (2500, 3000),    # 5元硬幣面積範圍
    10: (3500, 4000),   # 10元硬幣面積範圍
    50: (4500, 5000),   # 50元硬幣面積範圍
    20: (5200, 5800),   # 20元硬幣面積範圍
}


def recognize_coin(area):
    """
    根據輪廓面積判斷幣值。
    """
    for value, (min_a, max_a) in COIN_AREAS.items():
        if min_a <= area <= max_a:
            return value
    return None


def process_coins_by_contours(image_path):
    # 讀取影像
    src = cv2.imread(image_path)
    if src is None:
        print(f"錯誤：無法讀取影像，請確認檔案路徑: {image_path}")
        return

    output = src.copy()
    total_amount = 0

    # 1. 預處理：灰階和二值化
    gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # 2. 尋找輪廓
    # RETR_EXTERNAL 只會找最外層輪廓，忽略內部雜訊
    # CHAIN_APPROX_SIMPLE 壓縮輪廓點
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for i, contour in enumerate(contours):
        # 3. 面積篩選：排除過小的雜訊
        area = cv2.contourArea(contour)
        if area < 1000:  # 設定最小面積閾值，可根據情況調整
            continue
        
        # 4. 判斷幣值
        value = recognize_coin(area)

        if value is not None:
            total_amount += value
            color = COIN_COLORS.get(value, (255, 255, 255)) # 默認為白色

            # 計算輪廓的質心
            M = cv2.moments(contour)
            if M["m00"] != 0:
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
                
                # 繪製輪廓和幣值
                cv2.drawContours(output, [contour], -1, color, 2)
                cv2.putText(output, str(value), (cX, cY), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

    # 5. 在左上角顯示總金額
    cv2.putText(output, f"Total: {total_amount}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    
    # 顯示結果
    cv2.imshow("Result", output)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


# 調用函式並傳入你的圖片路徑
process_coins_by_contours("coins3.jpg")

In [1]:
import cv2
import numpy as np

def recognize_coin(radius):
    """
    根據半徑判斷幣值，從大到小進行判斷。
    """
    # 這些值是根據你的圖片所觀察到的數據進行估計的，請務必根據你的實際硬幣半徑進行微調
    if 55 <= radius <= 65:
        return 50 # 50元硬幣，半徑最大
    if 45 <= radius <= 54:
        return 10 # 10元硬幣
    if 35 <= radius <= 44:
        return 5  # 5元硬幣
    if 25 <= radius <= 34:
        return 1  # 1元硬幣
    
    return 0 # 沒有匹配到

def process_coins_final(image_path):
    # 讀取影像
    src = cv2.imread(image_path)
    if src is None:
        print(f"錯誤：無法讀取影像，請確認檔案路徑: {image_path}")
        return

    output = src.copy()

    # 預處理：灰階和高斯模糊
    gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (9, 9), 2)
    
    # 霍夫圓形變換，使用你已校準好的參數
    circles = cv2.HoughCircles(
        blurred,
        cv2.HOUGH_GRADIENT,
        dp=1.2,
        minDist=58,
        param1=73,
        param2=17,
        minRadius=24,
        maxRadius=57
    )
    
    total_amount = 0

    if circles is not None:
        circles = np.uint16(np.around(circles))
        
        for i in circles[0, :]:
            center_x, center_y, radius = i[0], i[1], i[2]
            
            # 判斷幣值
            value = recognize_coin(radius)
            
            if value > 0:
                total_amount += value
                
                # 繪製輪廓和幣值
                cv2.circle(output, (center_x, center_y), radius, (0, 255, 0), 2)
                cv2.putText(output, str(value), (center_x - 10, center_y + 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            else:
                # 無法辨識的硬幣用紅色標記
                cv2.circle(output, (center_x, center_y), radius, (0, 0, 255), 2)
                
    # 在圖片左上角顯示總金額
    cv2.putText(output, f"Total: {total_amount}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # 顯示結果
    cv2.imshow("Result", output)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


# 調用函式並傳入你的圖片路徑
process_coins_final("coins3.jpg")

In [10]:
import cv2
import numpy as np

def process_and_find_contours(image_path):
    """
    載入影像，進行預處理和輪廓檢測，並顯示所有輪廓。
    """
    # 讀取影像
    src = cv2.imread(image_path)
    if src is None:
        print(f"錯誤：無法讀取影像，請確認檔案路徑: {image_path}")
        return

    # 顯示原始影像
    cv2.imshow("Original Image", src)

    # 1. 轉換為灰階影像
    gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    
    # 2. 影像增強：直方圖均衡化
    # clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    # enhanced = clahe.apply(gray)
    enhanced = cv2.equalizeHist(gray)
    
    # 3. 自適應閾值
    # ADAPTIVE_THRESH_GAUSSIAN_C 更適合處理光線變化
    binary = cv2.adaptiveThreshold(enhanced, 255, 
                                   cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                   cv2.THRESH_BINARY_INV, 11, 2)
    
    # 4. 形態學操作：移除雜訊
    kernel = np.ones((5, 5), np.uint8)
    # 開運算：先侵蝕後膨脹，移除小的白色雜訊點
    opening = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel, iterations=1)
    # 閉運算：先膨脹後侵蝕，填補輪廓內部的小黑點或孔洞
    closing = cv2.morphologyEx(opening, cv2.MORPH_CLOSE, kernel, iterations=1)
    
    cv2.imshow("Processed Image", closing)
    
    # 5. 輪廓檢測
    contours, hierarchy = cv2.findContours(closing, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    output = src.copy()
    
    print(f"找到 {len(contours)} 個輪廓。")
    if not contours:
        print("沒有找到任何輪廓。請調整影像預處理步驟。")
    
    # 6. 繪製所有輪廓並顯示
    for i, contour in enumerate(contours):
        area = cv2.contourArea(contour)
        
        # 篩選掉過小的輪廓
        if area > 1000:  # 這裡的閾值需要根據你的影像來調整
            cv2.drawContours(output, contours, i, (0, 255, 0), 2)
            
            # 輔助：印出面積，讓你進行校準
            print(f"輪廓 {i} 的面積: {area}")
            
            # 標記輪廓編號
            M = cv2.moments(contour)
            if M["m00"] != 0:
                cX = int(M["m10"] / M["m00"])
                cY = int(M["m01"] / M["m00"])
                cv2.putText(output, str(i), (cX, cY), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)


    cv2.imshow("Detected Contours", output)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


# 使用你的圖片來運行程式碼
process_and_find_contours("coins3.jpg")

找到 39 個輪廓。


In [5]:
import cv2
import numpy as np

# 建立一個空函式，作為滑動條的回調函式
def nothing(x):
    pass

def calibrate_coins_radius(image_path):
    # 讀取影像
    src = cv2.imread(image_path)
    if src is None:
        print(f"錯誤：無法讀取影像，請確認檔案路徑: {image_path}")
        return

    # 縮小影像以加快處理速度，並提高圓形檢測準確性
    scale_factor = 0.5
    scaled_src = cv2.resize(src, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_AREA)

    # 創建一個視窗來顯示調整結果
    cv2.namedWindow('Coin Detection', cv2.WINDOW_AUTOSIZE)

    # 創建滑動條，用於調整參數
    cv2.createTrackbar('minRadius', 'Coin Detection', 30, 100, nothing)
    cv2.createTrackbar('maxRadius', 'Coin Detection', 70, 200, nothing)
    cv2.createTrackbar('param1', 'Coin Detection', 50, 200, nothing)
    cv2.createTrackbar('param2', 'Coin Detection', 25, 100, nothing)
    cv2.createTrackbar('minDist', 'Coin Detection', 80, 200, nothing)

    # 預處理：灰階和高斯模糊
    gray = cv2.cvtColor(scaled_src, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (9, 9), 2)

    while True:
        output = scaled_src.copy()

        # 讀取滑動條的值
        min_r = cv2.getTrackbarPos('minRadius', 'Coin Detection')
        max_r = cv2.getTrackbarPos('maxRadius', 'Coin Detection')
        param1 = cv2.getTrackbarPos('param1', 'Coin Detection')
        param2 = cv2.getTrackbarPos('param2', 'Coin Detection')
        minDist = cv2.getTrackbarPos('minDist', 'Coin Detection')
        
        # 進行霍夫圓形變換，使用滑動條的值
        circles = cv2.HoughCircles(
            blurred,
            cv2.HOUGH_GRADIENT,
            dp=1.2,
            minDist=minDist,
            param1=param1,
            param2=param2,
            minRadius=min_r,
            maxRadius=max_r
        )
        
        if circles is not None:
            circles = np.uint16(np.around(circles))
            print("--- 偵測到的硬幣半徑 ---")
            for i, coin in enumerate(circles[0, :]):
                center_x, center_y, radius = coin[0], coin[1], coin[2]
                
                # 繪製輪廓並標記半徑
                cv2.circle(output, (center_x, center_y), radius, (0, 255, 0), 2)
                cv2.putText(output, f"{radius:.2f}", (center_x - 10, center_y + 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
                
                # 印出半徑值
                print(f"硬幣 {i+1}: 半徑 = {radius:.2f}")
        
        # 顯示結果
        cv2.imshow('Coin Detection', output)
        
        # 按下 'q' 鍵退出迴圈
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()


# 調用函式並傳入你的圖片路徑
# 運行這個程式後，請觀察控制台輸出的半徑值，並根據這些值來更新你的程式碼
calibrate_coins_radius("coins3.jpg")

--- 偵測到的硬幣半徑 ---
硬幣 1: 半徑 = 45.00
硬幣 2: 半徑 = 38.00
硬幣 3: 半徑 = 46.00
硬幣 4: 半徑 = 47.00
硬幣 5: 半徑 = 41.00
硬幣 6: 半徑 = 35.00
硬幣 7: 半徑 = 39.00
硬幣 8: 半徑 = 40.00
硬幣 9: 半徑 = 31.00
硬幣 10: 半徑 = 38.00
--- 偵測到的硬幣半徑 ---
硬幣 1: 半徑 = 45.00
硬幣 2: 半徑 = 38.00
硬幣 3: 半徑 = 46.00
硬幣 4: 半徑 = 47.00
硬幣 5: 半徑 = 41.00
硬幣 6: 半徑 = 35.00
硬幣 7: 半徑 = 39.00
硬幣 8: 半徑 = 40.00
硬幣 9: 半徑 = 31.00
硬幣 10: 半徑 = 38.00
--- 偵測到的硬幣半徑 ---
硬幣 1: 半徑 = 45.00
硬幣 2: 半徑 = 38.00
硬幣 3: 半徑 = 46.00
硬幣 4: 半徑 = 47.00
硬幣 5: 半徑 = 41.00
硬幣 6: 半徑 = 35.00
硬幣 7: 半徑 = 39.00
硬幣 8: 半徑 = 40.00
硬幣 9: 半徑 = 31.00
硬幣 10: 半徑 = 38.00
--- 偵測到的硬幣半徑 ---
硬幣 1: 半徑 = 45.00
硬幣 2: 半徑 = 38.00
硬幣 3: 半徑 = 46.00
硬幣 4: 半徑 = 47.00
硬幣 5: 半徑 = 41.00
硬幣 6: 半徑 = 35.00
硬幣 7: 半徑 = 39.00
硬幣 8: 半徑 = 40.00
硬幣 9: 半徑 = 31.00
硬幣 10: 半徑 = 38.00
--- 偵測到的硬幣半徑 ---
硬幣 1: 半徑 = 45.00
硬幣 2: 半徑 = 38.00
硬幣 3: 半徑 = 46.00
硬幣 4: 半徑 = 47.00
硬幣 5: 半徑 = 41.00
硬幣 6: 半徑 = 35.00
硬幣 7: 半徑 = 39.00
硬幣 8: 半徑 = 40.00
硬幣 9: 半徑 = 31.00
硬幣 10: 半徑 = 38.00
--- 偵測到的硬幣半徑 ---
硬幣 1: 半徑 = 45.00
硬幣 2: 半徑 = 38.00
硬幣 3: 半徑 

In [14]:
import cv2
import numpy as np

# 建立一個空函式，作為滑動條的回調函式
def nothing(x):
    pass

def process_coins_with_threshold(image_path):
    # 讀取影像
    src = cv2.imread(image_path)
    if src is None:
        print(f"錯誤：無法讀取影像，請確認檔案路徑: {image_path}")
        return

    # 縮小圖片以優化處理
    src = cv2.resize(src, None, fx=0.2, fy=0.2)
    output = src.copy()

    # 影像轉成灰階
    src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    
    # 創建一個視窗來顯示調整結果
    cv2.namedWindow('Threshold Adjustment', cv2.WINDOW_AUTOSIZE)
    
    # 創建滑動條，用於調整二值化閾值
    cv2.createTrackbar('Threshold', 'Threshold Adjustment', 127, 255, nothing)

    while True:
        output = src.copy()
        
        # 讀取滑動條的值
        threshold_value = cv2.getTrackbarPos('Threshold', 'Threshold Adjustment')
        
        # 二值化處理影像，這是找出輪廓的關鍵
        _, dst_binary = cv2.threshold(src_gray, threshold_value, 255, cv2.THRESH_BINARY_INV)

        # 找尋影像內的輪廓
        contours, _ = cv2.findContours(dst_binary,
                                       cv2.RETR_EXTERNAL,
                                       cv2.CHAIN_APPROX_SIMPLE)
        
        print(f"在當前閾值 {threshold_value} 下找到 {len(contours)} 個輪廓。")

        for i, contour in enumerate(contours):
            # 簡單篩選掉過小的雜訊
            if cv2.contourArea(contour) > 500: # 這個值可以根據需要調整
                # 繪製輪廓
                cv2.drawContours(output, [contour], -1, (0, 255, 0), 2)
                
        # 在左上角顯示輪廓數量
        cv2.putText(output, f"Contours: {len(contours)}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                
        cv2.imshow("Threshold Adjustment", output)
        
        # 顯示二值化後的影像，方便觀察
        cv2.imshow("Binary Image", dst_binary)
        
        # 按下 'q' 鍵退出迴圈
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()


# 調用函式並傳入你的圖片路徑
# 運行程式，並調整滑動條，直到所有硬幣都被綠色框框住
process_coins_with_threshold("coins.jpg")

在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 1420 個輪廓。
在當前閾值 127 下找到 14

In [19]:
import cv2
import numpy as np

# 幣值與顏色的對應字典 (BGR 格式)
COIN_COLORS = {
    50: (0, 255, 255),  # 50元硬幣 (黃色)
    10: (255, 0, 0),    # 10元硬幣 (藍色)
    5: (0, 255, 0),     # 5元硬幣 (綠色)
    1: (0, 0, 255),     # 1元硬幣 (紅色)
}

# 建立一個空函式，作為滑動條的回調函式
def nothing(x):
    pass

def process_coins_hybrid(image_path):
    src = cv2.imread(image_path)
    if src is None:
        print(f"錯誤：無法讀取影像，請確認檔案路徑: {image_path}")
        return

    # 縮小圖片以優化處理
    src_resized = cv2.resize(src, None, fx=0.2, fy=0.2)
    
    # 影像轉成灰階
    src_gray = cv2.cvtColor(src_resized, cv2.COLOR_BGR2GRAY)
    
    # 創建一個視窗來顯示調整結果
    cv2.namedWindow('Contour & Circle Detection', cv2.WINDOW_AUTOSIZE)

    # 創建滑動條，用於調整輪廓和圓形檢測的參數
    cv2.createTrackbar('Contour_Thresh', 'Contour & Circle Detection', 100, 255, nothing)
    cv2.createTrackbar('Circle_minDist', 'Contour & Circle Detection', 50, 200, nothing)
    cv2.createTrackbar('Circle_param1', 'Contour & Circle Detection', 50, 200, nothing)
    cv2.createTrackbar('Circle_param2', 'Contour & Circle Detection', 20, 100, nothing)
    cv2.createTrackbar('Circle_minR', 'Contour & Circle Detection', 20, 100, nothing)
    cv2.createTrackbar('Circle_maxR', 'Contour & Circle Detection', 60, 200, nothing)

    while True:
        output = src_resized.copy()

        # 讀取滑動條的值
        contour_thresh_val = cv2.getTrackbarPos('Contour_Thresh', 'Contour & Circle Detection')
        circle_min_dist = cv2.getTrackbarPos('Circle_minDist', 'Contour & Circle Detection')
        circle_param1 = cv2.getTrackbarPos('Circle_param1', 'Contour & Circle Detection')
        circle_param2 = cv2.getTrackbarPos('Circle_param2', 'Contour & Circle Detection')
        circle_min_r = cv2.getTrackbarPos('Circle_minR', 'Contour & Circle Detection')
        circle_max_r = cv2.getTrackbarPos('Circle_maxR', 'Contour & Circle Detection')
        
        # --- 方法一：基於輪廓（Contour）的檢測 ---
        # 簡單的二值化處理
        _, dst_binary = cv2.threshold(src_gray, contour_thresh_val, 255, cv2.THRESH_BINARY_INV)
        contours, _ = cv2.findContours(dst_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            area = cv2.contourArea(contour)
            if area > 1000:  # 篩選掉小面積雜訊
                # 繪製輪廓
                cv2.drawContours(output, [contour], -1, (0, 0, 255), 2)
        
        # --- 方法二：基於霍夫圓形（HoughCircles）的檢測 ---
        blurred = cv2.GaussianBlur(src_gray, (9, 9), 2)
        circles = cv2.HoughCircles(blurred,
                                   cv2.HOUGH_GRADIENT,
                                   dp=1.2,
                                   minDist=circle_min_dist,
                                   param1=circle_param1,
                                   param2=circle_param2,
                                   minRadius=circle_min_r,
                                   maxRadius=circle_max_r)

        if circles is not None:
            circles = np.uint16(np.around(circles))
            for i in circles[0, :]:
                center_x, center_y, radius = i[0], i[1], i[2]
                
                # 繪製圓形
                cv2.circle(output, (center_x, center_y), radius, (255, 0, 0), 2)
        
        # 顯示結果
        cv2.imshow('Contour & Circle Detection', output)
        
        # 按下 'q' 鍵退出迴圈
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()


# 調用函式並傳入你的圖片路徑
process_coins_hybrid("coins.jpg")

In [2]:
import cv2
import numpy as np

# 建立一個空函式，作為滑動條的回調函式
def nothing(x):
    pass

def calibrate_hough_circles(image_path):
    # 讀取影像
    src = cv2.imread(image_path)
    if src is None:
        print(f"錯誤：無法讀取影像，請確認檔案路徑: {image_path}")
        return

    # 縮小圖片以優化處理
    src_resized = cv2.resize(src, None, fx=0.2, fy=0.2)
    
    # 影像轉成灰階並高斯模糊
    src_gray = cv2.cvtColor(src_resized, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(src_gray, (9, 9), 2)

    # 創建一個視窗來顯示調整結果
    cv2.namedWindow('HoughCircles Calibration', cv2.WINDOW_AUTOSIZE)

    # 創建滑動條，用於調整霍夫圓形檢測的參數
    cv2.createTrackbar('minDist', 'HoughCircles Calibration', 80, 200, nothing)
    cv2.createTrackbar('param1', 'HoughCircles Calibration', 50, 200, nothing)
    cv2.createTrackbar('param2', 'HoughCircles Calibration', 25, 100, nothing)
    cv2.createTrackbar('minRadius', 'HoughCircles Calibration', 30, 100, nothing)
    cv2.createTrackbar('maxRadius', 'HoughCircles Calibration', 70, 200, nothing)

    while True:
        output = src_resized.copy()

        # 讀取滑動條的值
        min_dist = cv2.getTrackbarPos('minDist', 'HoughCircles Calibration')
        param1 = cv2.getTrackbarPos('param1', 'HoughCircles Calibration')
        param2 = cv2.getTrackbarPos('param2', 'HoughCircles Calibration')
        min_r = cv2.getTrackbarPos('minRadius', 'HoughCircles Calibration')
        max_r = cv2.getTrackbarPos('maxRadius', 'HoughCircles Calibration')
        
        # 進行霍夫圓形變換
        circles = cv2.HoughCircles(
            blurred,
            cv2.HOUGH_GRADIENT,
            dp=1.2,
            minDist=min_dist,
            param1=param1,
            param2=param2,
            minRadius=min_r,
            maxRadius=max_r
        )

        if circles is not None:
            circles = np.uint16(np.around(circles))
            print("--- 偵測到的硬幣半徑 ---")
            for i, circle in enumerate(circles[0, :]):
                center_x, center_y, radius = circle[0], circle[1], circle[2]
                
                # 繪製圓形並標註半徑
                cv2.circle(output, (center_x, center_y), radius, (0, 255, 0), 2)
                cv2.putText(output, f"{radius:.2f}", (center_x - 10, center_y + 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
                
                # 印出半徑值
                print(f"硬幣 {i+1}: 半徑 = {radius:.2f}")
        
        # 顯示結果
        cv2.imshow('HoughCircles Calibration', output)
        
        # 按下 'q' 鍵退出迴圈
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()


# 調用函式並傳入你的圖片路徑
# 運行程式後，請觀察控制台輸出的半徑值，並根據這些值來更新你的程式碼
calibrate_hough_circles("coins.jpg")

--- 偵測到的硬幣半徑 ---
硬幣 1: 半徑 = 63.00
硬幣 2: 半徑 = 65.00
硬幣 3: 半徑 = 69.00
硬幣 4: 半徑 = 69.00
硬幣 5: 半徑 = 69.00
硬幣 6: 半徑 = 69.00
硬幣 7: 半徑 = 64.00
硬幣 8: 半徑 = 47.00
硬幣 9: 半徑 = 52.00
硬幣 10: 半徑 = 48.00
硬幣 11: 半徑 = 49.00
硬幣 12: 半徑 = 55.00
硬幣 13: 半徑 = 63.00
硬幣 14: 半徑 = 51.00
硬幣 15: 半徑 = 47.00
--- 偵測到的硬幣半徑 ---
硬幣 1: 半徑 = 63.00
硬幣 2: 半徑 = 65.00
硬幣 3: 半徑 = 69.00
硬幣 4: 半徑 = 69.00
硬幣 5: 半徑 = 69.00
硬幣 6: 半徑 = 69.00
硬幣 7: 半徑 = 64.00
硬幣 8: 半徑 = 47.00
硬幣 9: 半徑 = 52.00
硬幣 10: 半徑 = 48.00
硬幣 11: 半徑 = 49.00
硬幣 12: 半徑 = 55.00
硬幣 13: 半徑 = 63.00
硬幣 14: 半徑 = 51.00
硬幣 15: 半徑 = 47.00
--- 偵測到的硬幣半徑 ---
硬幣 1: 半徑 = 63.00
硬幣 2: 半徑 = 65.00
硬幣 3: 半徑 = 69.00
硬幣 4: 半徑 = 69.00
硬幣 5: 半徑 = 69.00
硬幣 6: 半徑 = 69.00
硬幣 7: 半徑 = 64.00
硬幣 8: 半徑 = 47.00
硬幣 9: 半徑 = 52.00
硬幣 10: 半徑 = 48.00
硬幣 11: 半徑 = 49.00
硬幣 12: 半徑 = 55.00
硬幣 13: 半徑 = 63.00
硬幣 14: 半徑 = 51.00
硬幣 15: 半徑 = 47.00
--- 偵測到的硬幣半徑 ---
硬幣 1: 半徑 = 63.00
硬幣 2: 半徑 = 65.00
硬幣 3: 半徑 = 69.00
硬幣 4: 半徑 = 69.00
硬幣 5: 半徑 = 69.00
硬幣 6: 半徑 = 69.00
硬幣 7: 半徑 = 64.00
硬幣 8: 半徑 = 47.00
硬幣 9: 半徑 = 52

In [3]:
import cv2
import numpy as np

# 幣值與顏色的對應字典 (BGR 格式)
COIN_COLORS = {
    50: (0, 255, 255),  # 50元硬幣 (黃色)
    10: (255, 0, 0),    # 10元硬幣 (藍色)
    5: (0, 255, 0),     # 5元硬幣 (綠色)
    1: (0, 0, 255),     # 1元硬幣 (紅色)
}

# 使用你找到的最佳參數
BEST_HOUGH_PARAMS = {
    'minDist': 157,
    'param1': 47,
    'param2': 19,
    'minRadius': 94,
    'maxRadius': 101,
}

def get_coin_value(avg_radius):
    """
    根據平均半徑來判斷幣值。
    這裡需要你手動設定，根據你分組的結果填入。
    例如：
    if 98 <= avg_radius <= 101:
        return 5
    elif 94 <= avg_radius <= 97:
        return 10
    else:
        return 0
    """
    # 這裡的邏輯需要你根據圖片 b0aa7b 的實際輸出進行填寫
    # 請根據實際情況替換這些值
    if 98 <= avg_radius <= 101:
        return 5
    elif 94 <= avg_radius <= 97:
        return 10
    else:
        return 0

def process_coins_final(image_path):
    src = cv2.imread(image_path)
    if src is None:
        print(f"錯誤：無法讀取影像，請確認檔案路徑: {image_path}")
        return

    # 縮小圖片以優化處理
    src_resized = cv2.resize(src, None, fx=0.2, fy=0.2)
    output = src_resized.copy()

    # 影像轉成灰階並高斯模糊
    src_gray = cv2.cvtColor(src_resized, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(src_gray, (9, 9), 2)
    
    # 使用你找到的最佳霍夫圓形參數
    circles = cv2.HoughCircles(
        blurred,
        cv2.HOUGH_GRADIENT,
        dp=1.2,
        minDist=BEST_HOUGH_PARAMS['minDist'],
        param1=BEST_HOUGH_PARAMS['param1'],
        param2=BEST_HOUGH_PARAMS['param2'],
        minRadius=BEST_HOUGH_PARAMS['minRadius'],
        maxRadius=BEST_HOUGH_PARAMS['maxRadius']
    )

    total_amount = 0
    
    if circles is not None:
        circles = np.uint16(np.around(circles))
        
        # 根據半徑對硬幣進行分組
        radius_groups = {}
        for circle in circles[0, :]:
            radius = circle[2]
            found_group = False
            for avg_r, group in radius_groups.items():
                # 如果半徑在相近範圍內，就將其歸入同一組
                if abs(radius - avg_r) < 3: # 這裡的3是容錯範圍，可調整
                    group.append(circle)
                    found_group = True
                    break
            if not found_group:
                radius_groups[radius] = [circle]
        
        # 處理每一組硬幣
        for avg_r, group in radius_groups.items():
            value = get_coin_value(avg_r)
            if value > 0:
                for circle in group:
                    center_x, center_y, radius = circle[0], circle[1], circle[2]
                    color = COIN_COLORS.get(value, (255, 255, 255))
                    
                    # 繪製圓形並標註幣值
                    cv2.circle(output, (center_x, center_y), radius, color, 2)
                    cv2.putText(output, str(value), (center_x - 10, center_y + 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
                                
                    total_amount += value

    # 在左上角顯示總金額
    cv2.putText(output, f"Total: {total_amount}", (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                
    cv2.imshow("Result", output)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


# 調用函式並傳入你的圖片路徑
process_coins_final("coins.jpg")

C:\Users\q5j6j\AppData\Local\Temp\ipykernel_16480\3890373613.py:80: RuntimeWarning: overflow encountered in scalar subtract
  if abs(radius - avg_r) < 3: # 這裡的3是容錯範圍，可調整


In [5]:
import cv2
import numpy as np

# 硬幣幣值與顏色的對應字典 (BGR 格式)
COIN_COLORS = {
    50: (0, 255, 255),  # 50元硬幣 (黃色)
    10: (255, 0, 0),    # 10元硬幣 (藍色)
    5: (0, 255, 0),     # 5元硬幣 (綠色)
    1: (0, 0, 255),     # 1元硬幣 (紅色)
}

# 硬幣 Hu 矩模板庫
# 這些值需要你根據你的硬幣圖片來生成。
# 這裡僅為範例值，你需要自己運行一次程式來獲取這些值。
HU_MOMENTS_TEMPLATES = {
    1: -6.5,  # 1元硬幣的 Hu 矩範例
    5: -5.8,  # 5元硬幣的 Hu 矩範例
    10: -5.5, # 10元硬幣的 Hu 矩範例
    50: -5.3, # 50元硬幣的 Hu 矩範例
}

def get_hu_moment_value(moments):
    """
    將 Hu 矩轉換為單一的可比對數值。
    """
    hu_moments_log = -np.sign(moments) * np.log10(np.abs(moments))
    return hu_moments_log[0]

def recognize_coin_by_hu_moment(hu_moment_val):
    """
    根據 Hu 矩值與模板庫進行比對，判斷幣值。
    """
    best_match = 0
    min_diff = float('inf')
    
    for value, template_val in HU_MOMENTS_TEMPLATES.items():
        diff = abs(hu_moment_val - template_val)
        if diff < min_diff:
            min_diff = diff
            best_match = value
            
    return best_match

def nothing(x):
    pass

def process_coins_with_manual_calibration(image_path):
    src = cv2.imread(image_path)
    if src is None:
        print(f"錯誤：無法讀取影像，請確認檔案路徑: {image_path}")
        return

    # 1. 縮小圖片至 0.2 的比例
    src_resized = cv2.resize(src, None, fx=0.1, fy=0.1)
    src_gray = cv2.cvtColor(src_resized, cv2.COLOR_BGR2GRAY)
    
    cv2.namedWindow('Calibration', cv2.WINDOW_AUTOSIZE)
    
    # 兩種二值化方法的參數
    cv2.createTrackbar('Adaptive_BlockSize', 'Calibration', 21, 100, nothing)
    cv2.createTrackbar('Adaptive_C', 'Calibration', 10, 50, nothing)
    cv2.createTrackbar('Global_Threshold', 'Calibration', 127, 255, nothing)
    
    # 形態學和輪廓過濾參數
    cv2.createTrackbar('Morph_Kernel', 'Calibration', 5, 15, nothing)
    cv2.createTrackbar('Area_Min', 'Calibration', 1000, 5000, nothing)
    
    # 圓形形狀比對參數
    cv2.createTrackbar('Shape_Match_Method', 'Calibration', 0, 2, nothing)
    
    # 建立一個圓形的輪廓模板
    circle_template = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (50, 50))
    contours_circle, _ = cv2.findContours(circle_template, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    while True:
        output = src_resized.copy()
        
        # 讀取滑動條的值
        block_size = cv2.getTrackbarPos('Adaptive_BlockSize', 'Calibration')
        if block_size % 2 == 0:
            block_size += 1
            
        c_val = cv2.getTrackbarPos('Adaptive_C', 'Calibration')
        global_thresh = cv2.getTrackbarPos('Global_Threshold', 'Calibration')
        morph_kernel = cv2.getTrackbarPos('Morph_Kernel', 'Calibration')
        area_min = cv2.getTrackbarPos('Area_Min', 'Calibration')
        shape_match_method = cv2.getTrackbarPos('Shape_Match_Method', 'Calibration')
        
        # 2. 雙重二值化處理
        binary1 = cv2.adaptiveThreshold(src_gray, 255,
                                        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                        cv2.THRESH_BINARY_INV,
                                        block_size, c_val)
        _, binary2 = cv2.threshold(src_gray, global_thresh, 255,
                                   cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        
        # 合併二值化結果
        final_binary = cv2.bitwise_or(binary1, binary2)

        # 3. 形態學操作
        kernel = np.ones((morph_kernel, morph_kernel), np.uint8)
        processed_binary = cv2.morphologyEx(final_binary, cv2.MORPH_OPEN, kernel, iterations=2)
        processed_binary = cv2.morphologyEx(processed_binary, cv2.MORPH_CLOSE, kernel, iterations=1)
        
        # 4. 輪廓檢測
        contours, _ = cv2.findContours(processed_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        total_amount = 0
        
        for contour in contours:
            area = cv2.contourArea(contour)
            
            if area > area_min:
                # 5. 圓形形狀比對，修正了錯誤的參數
                shape_match_score = cv2.matchShapes(contours_circle[0], contour, shape_match_method + 1, 0.0)
                
                # 這裡的閾值需要你根據你的測試結果來設定
                if shape_match_score < 0.2:
                    moments = cv2.moments(contour)
                    if moments["m00"] != 0:
                        hu_moments = cv2.HuMoments(moments)
                        hu_val = get_hu_moment_value(hu_moments)

                        value = recognize_coin_by_hu_moment(hu_val)
                        
                        cX = int(moments["m10"] / moments["m00"])
                        cY = int(moments["m01"] / moments["m00"])

                        if value > 0:
                            total_amount += value
                            color = COIN_COLORS.get(value, (255, 255, 255))
                            cv2.drawContours(output, [contour], -1, color, 2)
                            cv2.putText(output, str(value), (cX - 10, cY + 10),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
                        else:
                            cv2.drawContours(output, [contour], -1, (255, 255, 255), 2)
                            cv2.putText(output, "?", (cX - 10, cY + 10),
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

        cv2.putText(output, f"Total: {total_amount}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
                    
        cv2.imshow("Calibration", output)
        cv2.imshow("Binary Image", processed_binary)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()


# 調用函式並傳入你的圖片路徑
process_coins_with_manual_calibration("coins.jpg")

In [11]:
import cv2
import numpy as np

# 硬幣幣值與顏色的對應字典 (BGR 格式)
COIN_COLORS = {
    50: (0, 255, 255),
    10: (255, 0, 0),
    5: (0, 255, 0),
    1: (0, 0, 255),
}

# 硬幣 Hu 矩模板庫
# 這些值需要你根據你的硬幣圖片來生成。
# 這裡僅為範例值，你需要自己運行一次程式來獲取這些值。
HU_MOMENTS_TEMPLATES = {
    50: -5.3,
    10: -5.5,
    5: -5.8,
    1: -6.5,
}

def get_hu_moment_value(moments):
    """
    將 Hu 矩轉換為單一的可比對數值。
    """
    hu_moments_log = -np.sign(moments) * np.log10(np.abs(moments))
    return hu_moments_log[0]

def recognize_coin_by_hu_moment(hu_moment_val):
    """
    根據 Hu 矩值與模板庫進行比對，判斷幣值。
    """
    best_match = 0
    min_diff = float('inf')

    for value, template_val in HU_MOMENTS_TEMPLATES.items():
        diff = abs(hu_moment_val - template_val)
        if diff < 0.2:
            if diff < min_diff:
                min_diff = diff
                best_match = value

    return best_match

def nothing(x):
    pass

def calibrate_and_process_coins(image_path):
    src = cv2.imread(image_path)
    if src is None:
        print(f"錯誤：無法讀取影像，請確認檔案路徑: {image_path}")
        return

    src_resized = cv2.resize(src, None, fx=0.1, fy=0.1)
    output = src_resized.copy()
    src_gray = cv2.cvtColor(src_resized, cv2.COLOR_BGR2GRAY)

    cv2.namedWindow('Image Calibration', cv2.WINDOW_AUTOSIZE)

    # 1. 各種二值化方法的參數
    cv2.createTrackbar('Adaptive_BlockSize_G', 'Image Calibration', 21, 100, nothing)
    cv2.createTrackbar('Adaptive_C_G', 'Image Calibration', 10, 50, nothing)
    
    cv2.createTrackbar('Adaptive_BlockSize_M', 'Image Calibration', 21, 100, nothing)
    cv2.createTrackbar('Adaptive_C_M', 'Image Calibration', 10, 50, nothing)

    cv2.createTrackbar('Canny_Thresh1', 'Image Calibration', 50, 200, nothing)
    cv2.createTrackbar('Canny_Thresh2', 'Image Calibration', 150, 300, nothing)

    # 2. 形態學和輪廓過濾參數
    cv2.createTrackbar('Morph_Kernel', 'Image Calibration', 5, 15, nothing)
    cv2.createTrackbar('Area_Min', 'Image Calibration', 1000, 5000, nothing)
    cv2.createTrackbar('Area_Max', 'Image Calibration', 5000, 10000, nothing)
    
    while True:
        output = src_resized.copy()

        # 讀取滑動條值
        block_size_g = cv2.getTrackbarPos('Adaptive_BlockSize_G', 'Image Calibration')
        if block_size_g % 2 == 0: block_size_g += 1
        c_val_g = cv2.getTrackbarPos('Adaptive_C_G', 'Image Calibration')
        
        block_size_m = cv2.getTrackbarPos('Adaptive_BlockSize_M', 'Image Calibration')
        if block_size_m % 2 == 0: block_size_m += 1
        c_val_m = cv2.getTrackbarPos('Adaptive_C_M', 'Image Calibration')

        canny_thresh1 = cv2.getTrackbarPos('Canny_Thresh1', 'Image Calibration')
        canny_thresh2 = cv2.getTrackbarPos('Canny_Thresh2', 'Image Calibration')

        morph_kernel = cv2.getTrackbarPos('Morph_Kernel', 'Image Calibration')
        area_min = cv2.getTrackbarPos('Area_Min', 'Image Calibration')
        area_max = cv2.getTrackbarPos('Area_Max', 'Image Calibration')

        # 3. 多重二值化處理
        # Adaptive Gaussian Threshold
        binary_ada_g = cv2.adaptiveThreshold(src_gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, block_size_g, c_val_g)
        
        # Adaptive Mean Threshold
        binary_ada_m = cv2.adaptiveThreshold(src_gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, block_size_m, c_val_m)

        # Otsu's Threshold
        _, binary_otsu = cv2.threshold(src_gray, 175, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        
        # Canny Edge Detector (將邊緣轉換為白色，背景為黑色)
        edges = cv2.Canny(src_gray, canny_thresh1, canny_thresh2)

        # 合併所有二值化結果
        combined_binary = cv2.bitwise_or(binary_ada_g, binary_ada_m)
        combined_binary = cv2.bitwise_or(combined_binary, binary_otsu)
        combined_binary = cv2.bitwise_or(combined_binary, edges) # 合併 Canny 邊緣

        # 4. 形態學操作：開運算與閉運算
        kernel = np.ones((morph_kernel, morph_kernel), np.uint8)
        processed_binary = cv2.morphologyEx(combined_binary, cv2.MORPH_OPEN, kernel, iterations=2)
        processed_binary = cv2.morphologyEx(processed_binary, cv2.MORPH_CLOSE, kernel, iterations=1)

        # 5. 輪廓檢測
        contours, _ = cv2.findContours(processed_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        total_amount = 0

        for contour in contours:
            area = cv2.contourArea(contour)

            # 6. 多重過濾
            if not (area_min <= area <= area_max):
                continue

            (x, y), radius = cv2.minEnclosingCircle(contour)
            if radius == 0:
                continue
            # 長寬比檢查
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = float(w) / h
            if not (0.8 <= aspect_ratio <= 1.2): # 圓形長寬比接近1
                continue
            
            # 圓形度檢查 (面積與最小外接圓的比例)
            circle_area = np.pi * (radius ** 2)
            if not (0.7 <= (area / circle_area) <= 1.0): # 硬幣輪廓應該接近圓形
                continue

            # 7. Hu 矩比對
            moments = cv2.moments(contour)
            if moments["m00"] != 0:
                hu_moments = cv2.HuMoments(moments)
                hu_val = get_hu_moment_value(hu_moments)

                value = recognize_coin_by_hu_moment(hu_val)

                if value > 0:
                    total_amount += value
                    cX = int(moments["m10"] / moments["m00"])
                    cY = int(moments["m01"] / moments["m00"])
                    color = COIN_COLORS.get(value, (255, 255, 255))
                    cv2.drawContours(output, [contour], -1, color, 2)
                    cv2.putText(output, str(value), (cX - 10, cY + 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
                else:
                    cv2.drawContours(output, [contour], -1, (100, 100, 100), 2)

        cv2.putText(output, f"Total: {total_amount}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        cv2.imshow("Image Calibration", output)
        cv2.imshow("Binary Image (Combined)", processed_binary)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()

# 調用函式並傳入你的圖片路徑
calibrate_and_process_coins("coins.jpg")

In [4]:
import cv2
import numpy as np

# 硬幣幣值與顏色的對應字典 (BGR 格式)
COIN_COLORS = {
    50: (0, 255, 255),
    10: (255, 0, 0),
    5: (0, 255, 0),
    1: (0, 0, 255),
}

# 硬幣 Hu 矩模板庫
# 這些值需要你根據你的硬幣圖片來生成。
# 這裡僅為範例值，你需要自己運行一次程式來獲取這些值。
HU_MOMENTS_TEMPLATES = {
    50: -5.3,
    10: -5.5,
    5: -5.8,
    1: -6.5,
}

def get_hu_moment_value(moments):
    """將 Hu 矩轉換為單一的可比對數值。"""
    hu_moments_log = -np.sign(moments) * np.log10(np.abs(moments))
    return hu_moments_log[0]

def recognize_coin_by_hu_moment(hu_moment_val):
    """根據 Hu 矩值與模板庫進行比對，判斷幣值。"""
    best_match = 0
    min_diff = float('inf')

    for value, template_val in HU_MOMENTS_TEMPLATES.items():
        diff = abs(hu_moment_val - template_val)
        if diff < 0.2:
            if diff < min_diff:
                min_diff = diff
                best_match = value

    return best_match

def nothing(x):
    pass

def calibrate_and_process_coins(image_path):
    src = cv2.imread(image_path)
    if src is None:
        print(f"錯誤：無法讀取影像，請確認檔案路徑: {image_path}")
        return

    src_resized = cv2.resize(src, None, fx=0.2, fy=0.2)
    src_gray = cv2.cvtColor(src_resized, cv2.COLOR_BGR2GRAY)
    
    cv2.namedWindow('Image Calibration', cv2.WINDOW_AUTOSIZE)

    # 1. 各種二值化方法的參數
    cv2.createTrackbar('Adaptive_BlockSize', 'Image Calibration', 21, 100, nothing)
    cv2.createTrackbar('Adaptive_C', 'Image Calibration', 10, 50, nothing)
    cv2.createTrackbar('Canny_Thresh1', 'Image Calibration', 50, 200, nothing)
    cv2.createTrackbar('Canny_Thresh2', 'Image Calibration', 150, 300, nothing)

    # 2. 形態學和輪廓過濾參數
    cv2.createTrackbar('Morph_Kernel', 'Image Calibration', 5, 15, nothing)
    cv2.createTrackbar('Area_Min', 'Image Calibration', 1000, 5000, nothing)
    cv2.createTrackbar('Area_Max', 'Image Calibration', 5000, 10000, nothing)
    
    while True:
        output = src_resized.copy()

        # 讀取滑動條值
        block_size = cv2.getTrackbarPos('Adaptive_BlockSize', 'Image Calibration')
        if block_size % 2 == 0: block_size += 1
        c_val = cv2.getTrackbarPos('Adaptive_C', 'Image Calibration')
        
        canny_thresh1 = cv2.getTrackbarPos('Canny_Thresh1', 'Image Calibration')
        canny_thresh2 = cv2.getTrackbarPos('Canny_Thresh2', 'Image Calibration')

        morph_kernel = cv2.getTrackbarPos('Morph_Kernel', 'Image Calibration')
        area_min = cv2.getTrackbarPos('Area_Min', 'Image Calibration')
        area_max = cv2.getTrackbarPos('Area_Max', 'Image Calibration')

        # 3. 多重二值化處理
        # Adaptive Threshold (高斯法)
        binary_ada = cv2.adaptiveThreshold(src_gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, block_size, c_val)
        
        # Otsu's Threshold
        _, binary_otsu = cv2.threshold(src_gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
        
        # Canny Edge Detector
        edges = cv2.Canny(src_gray, canny_thresh1, canny_thresh2)

        # 合併所有二值化結果
        combined_binary = cv2.bitwise_or(binary_ada, binary_otsu)
        combined_binary = cv2.bitwise_or(combined_binary, edges)

        # 4. 形態學操作：開運算與閉運算
        kernel = np.ones((morph_kernel, morph_kernel), np.uint8)
        processed_binary = cv2.morphologyEx(combined_binary, cv2.MORPH_OPEN, kernel, iterations=2)
        processed_binary = cv2.morphologyEx(processed_binary, cv2.MORPH_CLOSE, kernel, iterations=1)

        # 5. 輪廓檢測
        contours, _ = cv2.findContours(processed_binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        total_amount = 0

        for contour in contours:
            area = cv2.contourArea(contour)

            # 6. 多重過濾
            if not (area_min <= area <= area_max):
                continue

            (x, y), radius = cv2.minEnclosingCircle(contour)
            if radius == 0:
                continue
            # 長寬比檢查
            x, y, w, h = cv2.boundingRect(contour)
            aspect_ratio = float(w) / h
            if not (0.8 <= aspect_ratio <= 1.2):
                continue
            
            # 圓形度檢查 (面積與最小外接圓的比例)
            circle_area = np.pi * (radius ** 2)
            if not (0.7 <= (area / circle_area) <= 1.0):
                continue

            # 7. Hu 矩比對
            moments = cv2.moments(contour)
            if moments["m00"] != 0:
                hu_moments = cv2.HuMoments(moments)
                hu_val = get_hu_moment_value(hu_moments)

                value = recognize_coin_by_hu_moment(hu_val)

                if value > 0:
                    total_amount += value
                    cX = int(moments["m10"] / moments["m00"])
                    cY = int(moments["m01"] / moments["m00"])
                    color = COIN_COLORS.get(value, (255, 255, 255))
                    cv2.drawContours(output, [contour], -1, color, 2)
                    cv2.putText(output, str(value), (cX - 10, cY + 10),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
                else:
                    cv2.drawContours(output, [contour], -1, (100, 100, 100), 2)

        cv2.putText(output, f"Total: {total_amount}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        cv2.imshow("Image Calibration", output)
        cv2.imshow("Binary Image (Combined)", processed_binary)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()

# 調用函式並傳入你的圖片路徑
calibrate_and_process_coins("coins.jpg")